In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# FCFS

*FCFS (First-Come, First-Served) is the simplest CPU scheduling algorithmwhere processes are executed in the order they arrive. It is non-preemptiveand follows the "first in, first out" approach. FCFS is used because it is easy to implement and ensures fairness, as everyprocess gets executed in turn without starvation. However, it may lead tohigh waiting times, especially if a long process arrives before shorter ones.Unlike algorithms like SJF (Shortest Job First) or Round Robin, FCFS does notconsider process length or priority, which can affect overall efficiency intime-critical systems.*



In [1]:
n = int(input("Enter number of processes: "))
at = [int(input(f"Enter arrival time for process {i + 1}: ")) for i in range(n)]
bt = [int(input(f"Enter burst time for process {i + 1}: ")) for i in range(n)]

wt = [0] * n
service_time = [0] * n
service_time[0] = at[0]

for i in range(1, n):
    service_time[i] = service_time[i - 1] + bt[i - 1]
    wt[i] = max(0, service_time[i] - at[i])

tat = [bt[i] + wt[i] for i in range(n)]
total_wt = sum(wt)
total_tat = sum(tat)

print("Processes Arrival Burst Waiting Turnaround")
for i in range(n):
    print(f"{i + 1}\t{at[i]}\t{bt[i]}\t{wt[i]}\t{tat[i]}")
print(f"\nAverage waiting time = {total_wt / n:.5f}")
print(f"Average turnaround time = {total_tat / n:.5f}")

Enter number of processes:  4
Enter arrival time for process 1:  0
Enter arrival time for process 2:  1
Enter arrival time for process 3:  2
Enter arrival time for process 4:  3
Enter burst time for process 1:  5
Enter burst time for process 2:  3
Enter burst time for process 3:  8
Enter burst time for process 4:  6


Processes Arrival Burst Waiting Turnaround
1	0	5	0	5
2	1	3	4	7
3	2	8	6	14
4	3	6	13	19

Average waiting time = 5.75000
Average turnaround time = 11.25000


-----------------

# SJF

SJF (Shortest Job First) is a CPU scheduling algorithm that selects the process
with the shortest burst time among the available processes at the current time.
It is non-preemptive, meaning once a process starts execution, it runs till
completion.

SJF minimizes the average waiting time and turnaround time but may lead to
starvation if short processes keep arriving and long ones are delayed. Unlike FCFS,
SJF considers burst time, making it more efficient in many cases, especially for
batch systems with known job lengths.
"""

In [4]:
n = int(input("Enter number of processes: "))
print()
at = [int(input(f"Enter arrival time for P{i + 1}: ")) for i in range(n)]
print()
bt = [int(input(f"Enter burst time for P{i + 1}: ")) for i in range(n)]

processes = list(range(n))
wt = [0] * n
tat = [0] * n
completion_time = [0] * n

remaining_processes = [(i, at[i], bt[i]) for i in range(n)]
current_time = min(at)
executed_processes = []

while remaining_processes:
    available = [p for p in remaining_processes if p[1] <= current_time]
    
    if not available:
        current_time = min(p[1] for p in remaining_processes)
        continue
    
    process = min(available, key=lambda x: x[2])
    pid = process[0]
    
    completion_time[pid] = current_time + process[2]
    
    tat[pid] = completion_time[pid] - at[pid]
    wt[pid] = tat[pid] - bt[pid]
    
    current_time = completion_time[pid]
    remaining_processes.remove(process)
    executed_processes.append(process)

total_wt = sum(wt)
total_tat = sum(tat)

print("\nProcesses | Arrival | Burst | Waiting | Turnaround")
print("-" * 50)
for i in range(n):
    print(f"{i + 1:^9} | {at[i]:^7} | {bt[i]:^5} | {wt[i]:^7} | {tat[i]:^10}")
print("-" * 50)
print(f"\nAverage waiting time = {total_wt / n:.2f}")
print(f"Average turnaround time = {total_tat / n:.2f}")

Enter number of processes:  4


Enter arrival time for P1:  0
Enter arrival time for P2:  1
Enter arrival time for P3:  2
Enter arrival time for P4:  3


Enter burst time for P1:  8
Enter burst time for P2:  4
Enter burst time for P3:  2
Enter burst time for P4:  1



Processes | Arrival | Burst | Waiting | Turnaround
--------------------------------------------------
    1     |    0    |   8   |    0    |     8     
    2     |    1    |   4   |   10    |     14    
    3     |    2    |   2   |    7    |     9     
    4     |    3    |   1   |    5    |     6     
--------------------------------------------------

Average waiting time = 5.50
Average turnaround time = 9.25


----------------------------

# SRTF

*SRTF (Shortest Remaining Time First) is a preemptive CPU scheduling algorithm that always selects the process with the shortest remaining burst time among the available processes at the current time. Unlike non-preemptive SJF, SRTF allows a new shorter process to preempt the currently running one.*

*This approach reduces average waiting and turnaround time even further compared to SJF, making it optimal in many scenarios. However, it may lead to starvation of longer processes if shorter ones keep arriving. SRTF is best suited for environments where process burst times are known or can be estimated accurately.*

In [1]:
n = int(input("Enter number of processes: "))
print()
at = [int(input(f"Enter arrival time for P{i + 1}: ")) for i in range(n)]
print()
bt = [int(input(f"Enter burst time for P{i + 1}: ")) for i in range(n)]

remaining_time = bt.copy()
completion_time = [0] * n
current_time, completed = 0, 0
wt, tat = [0] * n, [0] * n
is_completed = [False] * n

while completed < n:
    min_remaining = float('inf')
    selected_process = -1
    
    for i in range(n):
        if at[i] <= current_time and not is_completed[i]:
            if remaining_time[i] < min_remaining:
                min_remaining = remaining_time[i]
                selected_process = i
    
    if selected_process == -1:
        current_time += 1
        continue

    remaining_time[selected_process] -= 1

    if remaining_time[selected_process] == 0:
        completion_time[selected_process] = current_time + 1
        is_completed[selected_process] = True
        completed += 1

        tat[selected_process] = completion_time[selected_process] - at[selected_process]
        wt[selected_process] = tat[selected_process] - bt[selected_process]
    
    current_time += 1

total_wt = sum(wt)
total_tat = sum(tat)

print("\nProcesses | Arrival | Burst | Waiting | Turnaround")
print("-" * 50)
for i in range(n):
    print(f"{i + 1:^9} | {at[i]:^7} | {bt[i]:^5} | {wt[i]:^7} | {tat[i]:^10}")
print("-" * 50)
print(f"\nAverage waiting time = {total_wt / n:.2f}")
print(f"Average turnaround time = {total_tat / n:.2f}")

Enter number of processes:  4


Enter arrival time for P1:  0
Enter arrival time for P2:  1
Enter arrival time for P3:  2
Enter arrival time for P4:  3


Enter burst time for P1:  8
Enter burst time for P2:  4
Enter burst time for P3:  2
Enter burst time for P4:  1



Processes | Arrival | Burst | Waiting | Turnaround
--------------------------------------------------
    1     |    0    |   8   |    7    |     15    
    2     |    1    |   4   |    3    |     7     
    3     |    2    |   2   |    0    |     2     
    4     |    3    |   1   |    1    |     2     
--------------------------------------------------

Average waiting time = 2.75
Average turnaround time = 6.50


---------------------

# Round Robin Scheduling (RRS)

*Round Robin Scheduling (RRS) is a preemptive CPU scheduling algorithm designed for time-sharing systems. Each process is assigned a fixed time quantum in a cyclic order. If a process does not finish execution within its time quantum, it is placed at the end of the queue, and the CPU moves to the next process.*

*RRS ensures fair allocation of CPU time to all processes, preventing starvation and improving response time, especially for short jobs. However, if the time quantum is too small, it may lead to excessive context switching. If too large, it behaves like First-Come-First-Served (FCFS).*

*Choosing the right time quantum is key to achieving a balance between responsiveness and efficiency.*

In [10]:
from collections import deque

n = int(input('Enter number of processes: '))
at = list(map(int, input(f'Enter arrival times for {n} processes: ').split()))
bt = list(map(int, input(f'Enter burst times for {n} processes: ').split()))
tq = int(input('Enter time quantum: '))

remaining_bt = bt.copy()
wt = [0] * n
tat = [0] * n
ct = [0] * n
queue = deque()
t = 0
i = 0
visited = [False] * n

while i < n and at[i] <= t:
    queue.append(i)
    visited[i] = True
    i += 1

while queue:
    current = queue.popleft()
    exec_time = min(tq, remaining_bt[current])
    
    t += exec_time
    remaining_bt[current] -= exec_time

    if remaining_bt[current] > 0:
        queue.append(current)
    else:
        ct[current] = t
        tat[current] = ct[current] - at[current]
        wt[current] = tat[current] - bt[current]

    while i < n and at[i] <= t:
        if not visited[i]:
            queue.append(i)
            visited[i] = True
        i += 1

print('Process\tBurst Time\tWaiting Time\tTurnaround Time')
for i in range(n):
    print(f'{i+1}\t{bt[i]}\t\t{wt[i]}\t\t{tat[i]}')

avg_wt = sum(wt) / n
avg_tat = sum(tat) / n
print(f'Average Waiting Time: {avg_wt}')
print(f'Average Turnaround Time: {avg_tat}')

Enter number of processes:  4
Enter arrival times for 4 processes:  0 1 2 3
Enter burst times for 4 processes:  5 4 2 1
Enter time quantum:  2


Process	Burst Time	Waiting Time	Turnaround Time
1	5		4		9
2	4		7		11
3	2		4		6
4	1		6		7
Average Waiting Time: 5.25
Average Turnaround Time: 8.25


-----------

# Preemptive Priority Scheduling

*Preemptive Priority Scheduling is a CPU scheduling algorithm where each process is assigned a priority,  
and the CPU is allocated to the process with the highest priority (lower numerical value = higher priority).  
Unlike the non-preemptive version, this approach allows a higher-priority process to preempt a currently  
running lower-priority one as soon as it arrives.*

*This algorithm improves responsiveness for high-priority tasks but may cause starvation for lower-priority ones  
unless aging or other techniques are applied. It is more complex and requires efficient context switching mechanisms.*

*The program simulates real-time decision making at each unit of CPU time, executing the highest-priority  
available process, and calculating waiting time, turnaround time, and their averages.*

In [1]:
n = int(input("Enter number of processes: "))
print()
at = [int(input(f"Enter arrival time for P{i + 1}: ")) for i in range(n)]
print()
bt = [int(input(f"Enter burst time for P{i + 1}: ")) for i in range(n)]
print()
priority = [int(input(f"Enter priority for P{i + 1}: ")) for i in range(n)]

processes = list(range(n))
wt = [0] * n
tat = [0] * n
completion_time = [0] * n
remaining_bt = bt[:]

remaining_processes = [(i, at[i], bt[i], priority[i]) for i in range(n)]
current_time = 0
executed_processes = []

while remaining_processes:
    available = [p for p in remaining_processes if p[1] <= current_time]
    
    if not available:
        current_time = min(p[1] for p in remaining_processes)
        continue
    
    process = min(available, key=lambda x: (x[3], x[1]))
    pid = process[0]
    
    remaining_bt[pid] -= 1
    current_time += 1
    
    if remaining_bt[pid] == 0:
        completion_time[pid] = current_time
        tat[pid] = completion_time[pid] - at[pid]
        wt[pid] = tat[pid] - bt[pid]
        remaining_processes.remove(process)
        executed_processes.append(process)

total_wt = sum(wt)
total_tat = sum(tat)

print("\nProcesses | Arrival | Burst | Priority | Waiting | Turnaround")
print("-" * 60)
for i in range(n):
    print(f"{i + 1:^8} | {at[i]:^7} | {bt[i]:^5} | {priority[i]:^8} | {wt[i]:^7} | {tat[i]:^10}")
print("-" * 60)
print(f"\nAverage waiting time = {total_wt / n:.2f}")
print(f"Average turnaround time = {total_tat / n:.2f}")

Enter number of processes:  4


Enter arrival time for P1:  0
Enter arrival time for P2:  1
Enter arrival time for P3:  2
Enter arrival time for P4:  3


Enter burst time for P1:  8
Enter burst time for P2:  4
Enter burst time for P3:  9
Enter burst time for P4:  5


Enter priority for P1:  2
Enter priority for P2:  1
Enter priority for P3:  3
Enter priority for P4:  2



Processes | Arrival | Burst | Priority | Waiting | Turnaround
------------------------------------------------------------
   1     |    0    |   8   |    2     |    4    |     12    
   2     |    1    |   4   |    1     |    0    |     4     
   3     |    2    |   9   |    3     |   15    |     24    
   4     |    3    |   5   |    2     |    9    |     14    
------------------------------------------------------------

Average waiting time = 7.00
Average turnaround time = 13.50


------------

# Non-Preemptive Priority Scheduling

*Non-Preemptive Priority Scheduling is a CPU scheduling algorithm where each process is assigned a priority,  
and the CPU is allocated to the process with the highest priority (lower numerical value = higher priority).  
Unlike the preemptive version, once a process starts execution, it runs to completion—even if a higher  
priority process arrives during its execution.*

*This approach is simpler and incurs less overhead due to the absence of context switching. However, it can  
lead to higher average waiting times and response times for more urgent processes if they arrive after  
a lower-priority process has already started. It is best suited for batch systems or scenarios where  
preemption is undesirable.*

*The program simulates the execution of all processes by selecting from the available ones based on priority,  
calculating waiting time, turnaround time, and their averages for performance evaluation.*

In [2]:
n = int(input("Enter number of processes: "))
print()
at = [int(input(f"Enter arrival time for P{i + 1}: ")) for i in range(n)]
print()
bt = [int(input(f"Enter burst time for P{i + 1}: ")) for i in range(n)]
print()
priority = [int(input(f"Enter priority for P{i + 1}: ")) for i in range(n)]

processes = list(range(n))
wt = [0] * n
tat = [0] * n
completion_time = [0] * n

remaining_processes = [(i, at[i], bt[i], priority[i]) for i in range(n)]
current_time = min(at)
executed_processes = []

while remaining_processes:
    available = [p for p in remaining_processes if p[1] <= current_time]
    
    if not available:
        current_time = min(p[1] for p in remaining_processes)
        continue
    
    process = min(available, key=lambda x: (x[3], x[1]))
    pid = process[0]
    
    completion_time[pid] = current_time + process[2]
    
    tat[pid] = completion_time[pid] - at[pid]
    wt[pid] = tat[pid] - bt[pid]
    
    current_time = completion_time[pid]
    remaining_processes.remove(process)
    executed_processes.append(process)

total_wt = sum(wt)
total_tat = sum(tat)

print("\nProcesses | Arrival | Burst | Priority | Waiting | Turnaround")
print("-" * 60)
for i in range(n):
    print(f"{i + 1:^8} | {at[i]:^7} | {bt[i]:^5} | {priority[i]:^8} | {wt[i]:^7} | {tat[i]:^10}")
print("-" * 60)
print(f"\nAverage waiting time = {total_wt / n:.2f}")
print(f"Average turnaround time = {total_tat / n:.2f}")

Enter number of processes:  4


Enter arrival time for P1:  0
Enter arrival time for P2:  1
Enter arrival time for P3:  2
Enter arrival time for P4:  3


Enter burst time for P1:  5
Enter burst time for P2:  3
Enter burst time for P3:  8
Enter burst time for P4:  6


Enter priority for P1:  2
Enter priority for P2:  1
Enter priority for P3:  3
Enter priority for P4:  2



Processes | Arrival | Burst | Priority | Waiting | Turnaround
------------------------------------------------------------
   1     |    0    |   5   |    2     |    0    |     5     
   2     |    1    |   3   |    1     |    4    |     7     
   3     |    2    |   8   |    3     |   12    |     20    
   4     |    3    |   6   |    2     |    5    |     11    
------------------------------------------------------------

Average waiting time = 5.25
Average turnaround time = 10.75


--------------

# Producer-Consumer Problem Simulation

*This program simulates the Producer-Consumer problem using threading and a queue.*

*The `producer` thread generates and adds items to a shared queue, with a 1-second delay between each item.  
The `consumer` thread consumes items from the queue with a 2-second delay per item.*

*The queue is thread-safe, ensuring synchronization between producer and consumer. Both threads run concurrently,  
producing and consuming items while maintaining the queue's fixed size (5). Threads are managed using Python's `threading` module.*


In [ ]:
import queue
import threading
import time

buffer = queue.Queue(maxsize=5)

def producer():
    item = 1
    while True:
        buffer.put(item)
        print(f"Produced: {item}")
        item += 1
        time.sleep(1)

def consumer():
    while True:
        item = buffer.get()
        print(f"Consumed: {item}")
        time.sleep(2)

p = threading.Thread(target=producer)
c = threading.Thread(target=consumer)
p.start()
c.start()
p.join()
c.join()